In [33]:
def P(text): #  convert string to polynomial list  
    text = text.replace(' ','')
    text = text.replace('^-','~')
    terms0 = text.split('+')
    terms = []
    for i in range(len(terms0)):
        each = terms0[i]
        if '-' not in each:
            terms.append(each)
        else:
            new_each = each.split('-')
            terms.append(new_each[0])
            for j in range(len(new_each)-1):
                terms.append('-'+new_each[j+1])
    if '' in terms:
        terms.remove('')
            
    expanded = []
    for each in terms:
        term = term_convert(each)
        expanded.append(term)
    return simplify(expanded)

def term_convert(text):
    numbers = ['0','1','2','3','4','5','6','7','8','9','0','.','-','^','~','j']
    text = text.replace(' ','')
    splits = [0]
    for i in range(len(text)):
        cha = text[i]
        if cha not in numbers:
            splits.append(i)
    var = []
    for j in range(len(splits)-1):
        phrase = text[splits[j]:splits[j+1]]
        if phrase == '':
            var.append('1')
        else:
            var.append(phrase)
    var.append(text[splits[-1]:])
    expanded = []
    expanded.append(eval(var[0]))
    variables = []
    powers = []
    for i in range(1,len(var)):
        each = var[i]
        if '^' in each:
            split = each.split('^')
            variables.append(split[0])
            powers.append(eval(split[1]))
        elif '~' in each:
            split = each.split('~')
            variables.append(split[0])
            powers.append(-eval(split[1]))
        else:
            variables.append(each)
            powers.append(1)
        
    expanded.append(variables)
    expanded.append(list(powers))
    return expanded

    
def contract(collapsed):
    poly_out = ''
    for each in collapsed:
        coeff = str(each[0])
        poly_out += ' +'+ coeff
        for i in range(len(each[1])):
            poly_out += each[1][i] + '^' + str(each[2][i]) + '?'
    
    poly_out = poly_out.replace('(-0','')
    poly_out = poly_out.replace(')','')
            
    poly_out = poly_out.replace('+-','-')
    poly_out = poly_out.replace('^1?','')
    if poly_out[:2] == ' +':
        poly_out = poly_out[2:]
    else:
        poly_out = poly_out[1:]
    poly_out = poly_out.replace('?','')
    return poly_out
    
    
def multiply(term1,term2):
    term_out = []
    coeff = term1[0]*term2[0]
    term_out.append(coeff)
    var_pow = {}
    for i in range(len(term1[1])):
        if term1[1][i] in var_pow.keys():
            var_pow[term1[1][i]] += term1[2][i]
        else:
            var_pow[term1[1][i]] = term1[2][i]
    for i in range(len(term2[1])):
        if term2[1][i] in var_pow.keys():
            var_pow[term2[1][i]] += term2[2][i] 
        else:
            var_pow[term2[1][i]] = term2[2][i]
    variables = []
    powers = []
    for each in var_pow.keys():
        variables.append(each)
        powers.append(var_pow[each])
    term_out.append(variables)
    term_out.append(powers)
    return term_out


def add(P1,P2):
    added = [[0,[],[]]]
    for each in P1:
        added.append(each)
    for each in P2:
        added.append(each)   
    return simplify(added)

def addi(polys):
    added = polys[0]
    for i in range(1,len(polys)):
        added = add(added,polys[i])
    return simplify(added)
    

def mult(P1,P2):
    multiplied = [[0,[],[]]]
    for each in P1:
        for every in P2:
            multiplied.append(multiply(each,every))
    return simplify(multiplied)

def multi(polys):
    multiplied = polys[0]
    for i in range(1,len(polys)):
        multiplied = mult(multiplied,polys[i])
    return simplify(multiplied)

def split_complex(P):
    split_P = []
    for each in P:
        coeff = each[0]
        if type(coeff) != complex:
            split_P.append(each)
        else:
            if coeff.real != 0:
                split_P.append([coeff.real,each[1],each[2]])
            if coeff.imag != 0:
                split_P.append([float(coeff.imag)*1j,each[1],each[2]])
    return split_P

def remove_raised_0(P):
    kept = []
    for each in P:
        coeff = each[0]
        variables = []
        powers = []
        for i in range(len(each[1])):
            if each[2][i] != 0:
                variables.append(each[1][i])
                powers.append(each[2][i])
        kept.append([coeff,variables,powers])
    if kept == []:
        return [[0,[],[]]]
    return kept
    

def combine_like_terms(P):
    like_terms = {}
    for i in range(len(P)):
        term = P[i]
        alpha = term[1]
        powers = term[2]
        if len(alpha) > 1:
            alpha, powers = zip(*sorted(zip(alpha, powers)))
        alpha = list(alpha)
        powers = list(powers)
        T = str([alpha,powers])
        if T in like_terms:
            like_terms[T] += term[0]
        else:
            like_terms[T] = term[0]
    collapsed = []
    for each in like_terms.keys():
        coeff = like_terms[each]
        varpow = eval(each[1:-1])
        if type(coeff) == float and coeff == int(coeff):
            coeff = int(coeff)
        if type(coeff) == complex and coeff.imag == int(coeff.imag):
            coeff = int(coeff.imag)*1j
        if like_terms[each] != 0:
            collapsed.append([coeff,varpow[0],varpow[1]])
    if collapsed == []:
        return [[0,[],[]]]
    return collapsed

def simplify(P):
    P = split_complex(P)
    P = remove_raised_0(P)
    P = combine_like_terms(P)
    return P

def partial_term(var, P0):
    chain = {}
    func  = []
    
    for each in Chains.keys():
        if var == each:
            chain = Chains[var] 
            func = list(chain.keys())
            func.append(var) 
    chain[var] = [[1,[],[]]]
    func.sort()

    term = P0[0]
    cond = True
    for each in func:
        if each in term[1]:
            chosen_variable = each
            cond = False
            break
    if cond:
        return [[0,[],[]]]
    
    f_coeff = term[0]
    f_variables = []
    f_powers = []
    
    g_coeff = 1
    g_variables = []
    g_powers = []
    
    df_coeff = term[0]
    df_variables = []
    df_powers = []
    
    for i in range(len(term[1])):
        if term[1][i] == chosen_variable:
            f_variables.append(term[1][i])
            f_powers.append(term[2][i])
            
            df_coeff *= term[2][i]
            df_variables.append(term[1][i])
            df_powers.append(term[2][i]-1)
        else:
            g_variables.append(term[1][i])
            g_powers.append(term[2][i])
            
    f = [[f_coeff,f_variables,f_powers]]
    g = [[g_coeff,g_variables,g_powers]]
    
    P1 = [[df_coeff,df_variables,df_powers]]
    
    chain_rule = chain[chosen_variable]
    
    if type(chain_rule)== str:
        if '_' in chain_rule:
            chain_rule = [[1,[chain_rule],[1]]]
        else:
            chain_rule = P(chain_rule)
    
    df = mult(P1,chain_rule)
    
    return add(mult(g,df),mult(f,partial_term(var,g)))       



def partial(var, P):
    derivative = [[0,[],[]]]
    for term in P:
        term_derivative = partial_term(var,[term])
        for every in term_derivative:            
            derivative.append(every)
    return simplify(derivative)


def order(var,P,num,size):
    if type(var) != list:
        var = [var]
    if size == 'small':
        size = 1
    if size == 'large':
        size = -1
    expression = [[0,[],[]]]
    for each in P:
        Order = 0 
        for i in range(len(each[2])):
            if each[1][i] in var :
                Order += each[2][i]
        if not (Order*size > num):
            expression.append(each)
    return simplify(expression)

def substitute(P1,function,substitution): #can only substitute for positive integer powers 
    substituted = [[0,[],[]]]
    for term in P1:
        new_P = [term]
        if function in term[1]:
            power = term[2][term[1].index(function)]
            if not (power > 0 and type(power)==int):
                substituted = add(substituted,new_P)
                continue
            new_P = mult(new_P,[[1,[function],[-power]]])
            for j in range(power):
                new_P = mult(new_P,substitution)
        substituted = add(substituted,new_P)
    return simplify(substituted)


Chains = {'x':{ 'S':'1C','C':'-1S', 'e':'1e'}}

In [34]:
PS = P('1S')
PC = P('1C^-1')

PT = mult(PS,PC)

contract(partial('x',PT))

'1 +1C^-2S^2'

In [35]:
P1 = P('1jxy^2+5t')
P2 = P('y')
P3 = P('z+a^4')

P1

[[1j, ['x', 'y'], [1, 2]], [5, ['t'], [1]]]

In [36]:
contract(addi([P1,P2,P3]))

'1jxy^2 +5t +1y +1z +1a^4'

In [37]:
contract(mult(P1,P2))

'1jxy^3 +5ty'

In [38]:
contract(order(['a','x'],P3,3,'small'))

'1z'

In [39]:
P0= P('S^2')

contract(partial('x',partial('x',P0)))

'-2S^2 +2C^2'

In [40]:
x = P('-1jx+1x^-1+1+2')
y = P('-1.0jx+x^-1+3')
x == y

True

In [41]:
PT = P('x^7T^4e^5S^3')

contract(partial('x',PT))

'3CS^2T^4e^5x^7 +5S^3T^4e^5x^7 +7S^3T^4e^5x^6'

In [42]:
P0 = P('1S')

soln = P0

for i in range(101):
    soln = partial('x',soln)
    
contract(soln)

'1C'

In [43]:
def matrix_convert(Matrix):
    cond = False
    if type(Matrix[0][0]) == str:
        cond = True
    matrix = []
    for row in range(4):
        whole_row = []
        for column in range(4):
            if cond:
                whole_row.append(P(Matrix[row][column]))
            else:
                whole_row.append(contract(Matrix[row][column]))
        matrix.append(whole_row)
    return matrix 

def matrix_order(Order_list, Matrix, num, size):
    matrix = []
    for row in range(4):
        whole_row = []
        for column in range(4):
            whole_row.append(contract( order(Order_list,Matrix[row][column],num,size) ))
        matrix.append(whole_row)
    return matrix 
    
    
def christoffel(beta,mu,nu,metric,metric_con):
    x = list(Chains.keys())
    value = [[0,[],[]]]
    for a in range(4):
        paren1= partial(x[nu],metric[a][mu])
        paren2= partial(x[mu],metric[a][nu])
        paren3= mult(P('-1'),partial(x[a],metric[mu][nu]))
        paren = addi([paren1,paren2,paren3])
        value = add(value,multi([P('0.5'), metric_con[beta][a],paren]))
    return value

def Christoffel(metric,metric_con):
    not_a_tensor = []
    for depth in range(4):
        matrix = []
        for row in range(4):
            whole_row = []
            for column in range(4):
                whole_row.append(christoffel(depth,row,column,metric,metric_con))
            matrix.append(whole_row)
        not_a_tensor.append(matrix)
    return not_a_tensor

def ricci_tensor(mu,nu,metric,metric_con):
    x = list(Chains.keys())
    Christ_Symbol = Christoffel(metric,metric_con)
    
    first  = [[0,[],[]]]
    for a in range(4):
        first = add(first,partial(x[a],Christ_Symbol[a][mu][nu]))
    
    second = [[0,[],[]]]
    for a in range(4):
        second = add(second,partial(x[nu],Christ_Symbol[a][mu][a]))
    second = mult(P('-1'),second)
    
    third  = [[0,[],[]]]
    for b in range(4):
        for a in range(4):
            third = add(third,mult(Christ_Symbol[a][b][a],Christ_Symbol[b][mu][nu]))
    
    fourth = [[0,[],[]]]
    for b in range(4):
        for a in range(4):
            fourth = add(fourth,mult(Christ_Symbol[a][b][nu],Christ_Symbol[b][mu][a]))
    fourth = mult(P('-1'),fourth)
    
    return addi([first,second,third,fourth])

def Ricci_Tensor(metric,metric_con):
    matrix = []
    for row in range(4):
        whole_row = []
        for column in range(4):
            whole_row.append(ricci_tensor(row,column,metric,metric_con))
        matrix.append(whole_row)
    return matrix

def Ricci_Scalar(metric,metric_con):
    Tensor = Ricci_Tensor(metric,metric_con)
    scalar = [[0,[],[]]]
    for row in range(4):
        for column in range(4):
            scalar = add(scalar,mult(metric_con[row][column],Tensor[row][column]))
    return scalar

def Einstein_Tensor(R_Tensor,R_Scalar,metric):
    Tensor = []
    for i in range(4):
        row = []
        for j in range(4):
            row.append( add(R_Tensor[i][j],multi([P('-0.5'),metric[i][j],R_Scalar])))
        Tensor.append(row)
    return Tensor

# Any Spacially Equivalent Diagaonal Metric

In [44]:
A0 =[['f', '0', '0', '0'], \
    [ '0', 'g', '0', '0'], \
    [ '0', '0', 'g', '0'], \
    [ '0', '0', '0', 'g']]

A0_con =[['f^-1', '0', '0', '0'], \
    [ '0', 'g^-1', '0', '0'], \
    [ '0', '0', 'g^-1', '0'], \
    [ '0', '0', '0', 'g^-1']]

Chains = {'t':{'f':'f_t', 'g':'g_t', 'f_t':'f_tt', 'f_x':'f_tx', 'f_y':'f_ty' , 'f_z':'f_tz', 'g_t':'g_tt', 'g_x':'g_tx', 'g_y':'g_ty', 'g_z':'g_tz' }, \
          'x':{'f':'f_x', 'g':'g_x', 'f_t':'f_tx', 'f_x':'f_xx', 'f_y':'f_xy' , 'f_z':'f_xz', 'g_t':'g_tx', 'g_x':'g_xx', 'g_y':'g_xy', 'g_z':'g_xz' }, \
          'y':{'f':'f_y', 'g':'g_y', 'f_t':'f_ty', 'f_x':'f_xy', 'f_y':'f_yy' , 'f_z':'f_yz', 'g_t':'g_ty', 'g_x':'g_xy', 'g_y':'g_yy', 'g_z':'g_yz' }, \
          'z':{'f':'f_z', 'g':'g_z', 'f_t':'f_tz', 'f_x':'f_xz', 'f_y':'f_yz' , 'f_z':'f_zz', 'g_t':'g_tz', 'g_x':'g_xz', 'g_y':'g_yz', 'g_z':'g_zz' }}

A      = matrix_convert(A0)
A_con  = matrix_convert(A0_con)

In [45]:
matrix_convert(Christoffel(A,A_con)[0])

[['0.5f^-1f_t', '0.5f^-1f_x', '0.5f^-1f_y', '0.5f^-1f_z'],
 ['0.5f^-1f_x', '-0.5f^-1g_t', '0', '0'],
 ['0.5f^-1f_y', '0', '-0.5f^-1g_t', '0'],
 ['0.5f^-1f_z', '0', '0', '-0.5f^-1g_t']]

In [46]:
matrix_convert(Christoffel(A,A_con)[1])

[['-0.5f_xg^-1', '0.5g^-1g_t', '0', '0'],
 ['0.5g^-1g_t', '0.5g^-1g_x', '0.5g^-1g_y', '0.5g^-1g_z'],
 ['0', '0.5g^-1g_y', '-0.5g^-1g_x', '0'],
 ['0', '0.5g^-1g_z', '0', '-0.5g^-1g_x']]

In [47]:
matrix_convert(Christoffel(A,A_con)[2])

[['-0.5f_yg^-1', '0', '0.5g^-1g_t', '0'],
 ['0', '-0.5g^-1g_y', '0.5g^-1g_x', '0'],
 ['0.5g^-1g_t', '0.5g^-1g_x', '0.5g^-1g_y', '0.5g^-1g_z'],
 ['0', '0', '0.5g^-1g_z', '-0.5g^-1g_y']]

In [48]:
matrix_convert(Christoffel(A,A_con)[3])

[['-0.5f_zg^-1', '0', '0', '0.5g^-1g_t'],
 ['0', '-0.5g^-1g_z', '0', '0.5g^-1g_x'],
 ['0', '0', '-0.5g^-1g_z', '0.5g^-1g_y'],
 ['0.5g^-1g_t', '0.5g^-1g_x', '0.5g^-1g_y', '0.5g^-1g_z']]

In [49]:
matrix_convert(Ricci_Tensor(A,A_con))

[['-0.5f_xxg^-1 -0.25f_xg^-2g_x -0.5f_yyg^-1 -0.25f_yg^-2g_y -0.5f_zzg^-1 -0.25f_zg^-2g_z +0.75g^-2g_t^2 -1.5g^-1g_tt +0.75f^-1f_tg^-1g_t +0.25f^-1f_x^2g^-1 +0.25f^-1f_y^2g^-1 +0.25f^-1f_z^2g^-1',
  '1g^-2g_tg_x -1g^-1g_tx +0.5f^-1f_xg^-1g_t',
  '1g^-2g_tg_y -1g^-1g_ty +0.5f^-1f_yg^-1g_t',
  '1g^-2g_tg_z -1g^-1g_tz +0.5f^-1f_zg^-1g_t'],
 ['1g^-2g_tg_x -1g^-1g_tx +0.5f^-1f_xg^-1g_t',
  '0.25f^-2f_tg_t -0.5f^-1g_tt +1g^-2g_x^2 -1g^-1g_xx +0.25g^-2g_y^2 -0.5g^-1g_yy +0.25g^-2g_z^2 -0.5g^-1g_zz +0.25f^-2f_x^2 -0.5f^-1f_xx -0.25f^-1g^-1g_t^2 +0.25f^-1f_xg^-1g_x -0.25f^-1f_yg^-1g_y -0.25f^-1f_zg^-1g_z',
  '0.75g^-2g_xg_y -0.5g^-1g_xy +0.25f^-2f_xf_y -0.5f^-1f_xy +0.25f^-1f_xg^-1g_y +0.25f^-1f_yg^-1g_x',
  '0.75g^-2g_xg_z -0.5g^-1g_xz +0.25f^-2f_xf_z -0.5f^-1f_xz +0.25f^-1f_xg^-1g_z +0.25f^-1f_zg^-1g_x'],
 ['1g^-2g_tg_y -1g^-1g_ty +0.5f^-1f_yg^-1g_t',
  '0.75g^-2g_xg_y -0.5g^-1g_xy +0.25f^-2f_xf_y -0.5f^-1f_xy +0.25f^-1f_xg^-1g_y +0.25f^-1f_yg^-1g_x',
  '0.25f^-2f_tg_t -0.5f^-1g_tt +0.25g^-2g

In [50]:
contract(Ricci_Scalar(A,A_con))

'-1f^-1f_xxg^-1 -1f^-1f_yyg^-1 -0.5f^-1f_yg^-2g_y -1f^-1f_zzg^-1 -0.5f^-1f_zg^-2g_z -3f^-1g^-1g_tt +1.5f^-2f_tg^-1g_t +0.5f^-2f_x^2g^-1 +0.5f^-2f_y^2g^-1 +0.5f^-2f_z^2g^-1 +1.5g^-3g_x^2 -2g^-2g_xx +1.5g^-3g_y^2 -2g^-2g_yy +1.5g^-3g_z^2 -2g^-2g_zz -0.5f^-1f_xg^-2g_x'

In [51]:
RT = Ricci_Tensor(A,A_con)

In [52]:
contract(RT[0][0])

'-0.5f_xxg^-1 -0.25f_xg^-2g_x -0.5f_yyg^-1 -0.25f_yg^-2g_y -0.5f_zzg^-1 -0.25f_zg^-2g_z +0.75g^-2g_t^2 -1.5g^-1g_tt +0.75f^-1f_tg^-1g_t +0.25f^-1f_x^2g^-1 +0.25f^-1f_y^2g^-1 +0.25f^-1f_z^2g^-1'

# Flat Metric

In [22]:
g0 =[['-1', '0', '0', '0'], \
    [ '0', '1', '0', '0'], \
    [ '0', '0', '1', '0'], \
    [ '0', '0', '0', '1']]

Chains = {'t':{}, 'x':{}, 'y':{}, 'z':{}}

g      = matrix_convert(g0)
g_con  = matrix_convert(g0)

In [23]:
matrix_convert(Christoffel(g,g_con)[0])

[['0', '0', '0', '0'],
 ['0', '0', '0', '0'],
 ['0', '0', '0', '0'],
 ['0', '0', '0', '0']]

In [24]:
matrix_convert(Christoffel(g,g_con)[1])

[['0', '0', '0', '0'],
 ['0', '0', '0', '0'],
 ['0', '0', '0', '0'],
 ['0', '0', '0', '0']]

In [25]:
matrix_convert(Christoffel(g,g_con)[2])

[['0', '0', '0', '0'],
 ['0', '0', '0', '0'],
 ['0', '0', '0', '0'],
 ['0', '0', '0', '0']]

In [26]:
matrix_convert(Christoffel(g,g_con)[3])

[['0', '0', '0', '0'],
 ['0', '0', '0', '0'],
 ['0', '0', '0', '0'],
 ['0', '0', '0', '0']]

In [27]:
matrix_convert(Ricci_Tensor(g,g_con))

[['0', '0', '0', '0'],
 ['0', '0', '0', '0'],
 ['0', '0', '0', '0'],
 ['0', '0', '0', '0']]

In [28]:
contract(Ricci_Scalar(g,g_con))

'0'

# Scwarchild Metric

$$K \equiv 1 - \frac{2GM}{c^2r} \ \ \ \ \ \ \ \ \ \ K \equiv 1 - \frac{r_s}{r}$$

$$ S \equiv \sin(\theta) \ \ \ \ \ \ \ C \equiv \cos(\theta) $$

In [36]:

#K = [[1,[],[]],[-2,['G','M','c','r'],[1,1,-2,-1]]]

K = [[1,[],[]],[-1,['r','r_s'],[-1,1]]]

schwar0 =[[ '-1K', '0', '0', '0'], \
          [ '0', '1 K^-1', '0', '0'], \
          [ '0', '0', '1r^2', '0'], \
          [ '0', '0', '0', '1r^2S^2']]

schwar_con0 =[  ['-1K^-1', '0', '0', '0'], \
                [ '0', 'K' , '0', '0'], \
                [ '0', '0', '1r^-2', '0'], \
                [ '0', '0', '0', '1r^-2S^-2']]

# for mks
Chains_ = { \
             't':{},\
             'r':{ 'K':[[2,['G','M','c','r'],[1,1,-2,-2]]]  },\
             'theta':{ 'S':'C','C':'-1S' },\
             'phi':{}\
             }
# for c=1                    
Chains = { \
             't':{},\
             'r':{ 'K':partial('r',K)  },\
             'theta':{ 'S':'C','C':'-1S' },\
             'phi':{}\
             }

schwar     = matrix_convert(schwar0)
schwar_con = matrix_convert(schwar_con0)

In [37]:
matrix_convert(Christoffel(schwar,schwar_con)[0])

[['0', '0.5K^-1r^-2r_s', '0', '0'],
 ['0.5K^-1r^-2r_s', '0', '0', '0'],
 ['0', '0', '0', '0'],
 ['0', '0', '0', '0']]

In [38]:
matrix_convert(Christoffel(schwar,schwar_con)[1])

[['0.5Kr^-2r_s', '0', '0', '0'],
 ['0', '-0.5K^-1r^-2r_s', '0', '0'],
 ['0', '0', '-1Kr', '0'],
 ['0', '0', '0', '-1KS^2r']]

In [39]:
matrix_convert(Christoffel(schwar,schwar_con)[2])

[['0', '0', '0', '0'],
 ['0', '0', '1r^-1', '0'],
 ['0', '1r^-1', '0', '0'],
 ['0', '0', '0', '-1CS']]

In [40]:
matrix_convert(Christoffel(schwar,schwar_con)[3])

[['0', '0', '0', '0'],
 ['0', '0', '0', '1r^-1'],
 ['0', '0', '0', '1CS^-1'],
 ['0', '1r^-1', '1CS^-1', '0']]

In [41]:
matrix_convert(Ricci_Tensor(schwar,schwar_con))

[['0', '0', '0', '0'],
 ['0', '0', '0', '0'],
 ['0', '0', '1 -1r^-1r_s -1K', '0'],
 ['0', '0', '0', '-1S^2r^-1r_s -1KS^2 +1S^2']]

In [42]:
YY = ricci_tensor(2,2,schwar,schwar_con)

contract(substitute(YY,'K',K))

'0'

In [46]:
ZZ = ricci_tensor(3,3,schwar,schwar_con)

contract(substitute(ZZ,'K',K))

'0'

In [47]:
R = Ricci_Scalar(schwar,schwar_con)
contract(R)

'2r^-2 -2r^-3r_s -2Kr^-2'

In [48]:
contract(substitute(R,'K',K))

'0'

# Einstein and Rosen

$$ K \equiv 1 - \frac{r_s}{r} - \frac{\epsilon^2}{2r^2} $$

In [49]:
Kw = [[1,[],[]],[-1,['r','r_s'],[-1,1]],[-0.5,['e','r'],[2,-2]]]
Chains = { \
             't':{},\
             'r':{ 'K':partial('r',Kw)  },\
             'theta':{ 'S':'C','C':'-1S' },\
             'phi':{}\
             }

In [50]:
R = Ricci_Scalar(schwar,schwar_con)
contract(R)


'2r^-2 -2r^-3r_s -2Kr^-2 -1e^2r^-4'

In [51]:
contract(substitute(R,'K',Kw))

'0'

# Friedman-Robertson-Walker

In [52]:
#c = 1
frw0 = [['-1', '0', '0', '0'], \
        [ '0', 'a^2', '0', '0'], \
        [ '0', '0', 'a^2', '0'], \
        [ '0', '0', '0', 'a^2']]

frw_con0 = [['-1', '0', '0', '0'], \
            [ '0', 'a^-2', '0', '0'], \
            [ '0', '0', 'a^-2', '0'], \
            [ '0', '0', '0', 'a^-2']]


Chains = { \
             't':{'a':'a_t',    'a_t':'a_tt'},\
             'x':{ },\
             'y':{ },\
             'z':{ } \
             }

frw     = matrix_convert(frw0)
frw_con = matrix_convert(frw_con0)

In [53]:
matrix_convert(Christoffel(frw,frw_con)[0])

[['0', '0', '0', '0'],
 ['0', '1aa_t', '0', '0'],
 ['0', '0', '1aa_t', '0'],
 ['0', '0', '0', '1aa_t']]

In [54]:
matrix_convert(Christoffel(frw,frw_con)[1])

[['0', '1a^-1a_t', '0', '0'],
 ['1a^-1a_t', '0', '0', '0'],
 ['0', '0', '0', '0'],
 ['0', '0', '0', '0']]

In [55]:
matrix_convert(Christoffel(frw,frw_con)[2])

[['0', '0', '1a^-1a_t', '0'],
 ['0', '0', '0', '0'],
 ['1a^-1a_t', '0', '0', '0'],
 ['0', '0', '0', '0']]

In [56]:
matrix_convert(Christoffel(frw,frw_con)[3])

[['0', '0', '0', '1a^-1a_t'],
 ['0', '0', '0', '0'],
 ['0', '0', '0', '0'],
 ['1a^-1a_t', '0', '0', '0']]

In [57]:
matrix_convert(Ricci_Tensor(frw,frw_con))

[['-3a^-1a_tt', '0', '0', '0'],
 ['0', '2a_t^2 +1aa_tt', '0', '0'],
 ['0', '0', '2a_t^2 +1aa_tt', '0'],
 ['0', '0', '0', '2a_t^2 +1aa_tt']]

In [58]:
scalar_string0 = contract(Ricci_Scalar(frw,frw_con))
scalar_string0

'6a^-1a_tt +6a^-2a_t^2'

In [59]:
upper = 1000

scalar_string = scalar_string0

for power in range(upper):
    scalar_string = scalar_string.replace('^{}'.format(power),'^[{}]'.format(power))
    scalar_string = scalar_string.replace('^-{}'.format(power),'^[-{}]'.format(power))
scalar_string = scalar_string.replace('[','{')  
scalar_string = scalar_string.replace(']','}')

scalar_string = scalar_string.replace('a_tt','QQ7QQ')
scalar_string = scalar_string.replace('a_t','QQ8QQ')
scalar_string = scalar_string.replace('a','QQ9QQ')

scalar_string = scalar_string.replace('QQ7QQ','\ddot{a}')
scalar_string = scalar_string.replace('QQ8QQ','\dot{a}')
scalar_string = scalar_string.replace('QQ9QQ','a')

print('$$'+ scalar_string +'$$')

$$6a^{-1}\ddot{a} +6a^{-2}\dot{a}^{2}$$


$$6a^{-1}\ddot{a} +6a^{-2}\dot{a}^{2}$$

# Friedman-Robertson-Walker Perturbed

$$ S \equiv \Psi \ \ \ P \equiv \Phi $$

In [60]:
frw0 = [['-1-2S', '0', '0', '0'], \
        [ '0', 'a^2+2a^2P', '0', '0'], \
        [ '0', '0', 'a^2+2a^2P', '0'], \
        [ '0', '0', '0', 'a^2+2a^2P']]

frw_con0 = [['-1+2S', '0', '0', '0'], \
            [ '0', 'a^-2 -2a^-2 P', '0', '0'], \
            [ '0', '0', 'a^-2 -2a^-2 P', '0'], \
            [ '0', '0', '0', 'a^-2 -2a^-2 P']]


Chains = { \
             't':{'a':'a_t',    'a_t':'a_tt',\
                  'S':'S_t',    'S_t':'S_tt',\
                  'P':'P_t',    'P_t':'P_tt'},\
             'x':{ 'S':[[1j,['k_x','S'],[1,1]]],  'S_t':[[1j,['k_x','S_t'],[1,1]]],\
                   'P':[[1j,['k_x','P'],[1,1]]],  'P_t':[[1j,['k_x','P_t'],[1,1]]]},\
             'y':{ 'S':[[1j,['k_y','S'],[1,1]]],  'S_t':[[1j,['k_y','S_t'],[1,1]]],\
                   'P':[[1j,['k_y','P'],[1,1]]],  'S_t':[[1j,['k_y','P_t'],[1,1]]]},\
             'z':{ 'S':[[1j,['k_z','S'],[1,1]]],  'S_t':[[1j,['k_z','S_t'],[1,1]]],\
                   'P':[[1j,['k_z','P'],[1,1]]],  'P_t':[[1j,['k_z','P_t'],[1,1]]]} \
             }

order_variables = ['S','S_t','S_tt','P','P_t','P_tt']


frw     = matrix_convert(frw0)
frw_con = matrix_convert(frw_con0)

In [61]:
matrix_order(order_variables,Christoffel(frw,frw_con)[0],1,'small')

[['1S_t', '1jSk_x', '1jSk_y', '1jSk_z'],
 ['1jSk_x', '1aa_t +1P_ta^2 +2Paa_t -2Saa_t', '0', '0'],
 ['1jSk_y', '0', '1aa_t +1P_ta^2 +2Paa_t -2Saa_t', '0'],
 ['1jSk_z', '0', '0', '1aa_t +1P_ta^2 +2Paa_t -2Saa_t']]

In [62]:
matrix_order(order_variables,Christoffel(frw,frw_con)[1],1,'small')

[['1jSa^-2k_x', '1a^-1a_t +1P_t', '0', '0'],
 ['1a^-1a_t +1P_t', '1jPk_x', '1jPk_y', '1jPk_z'],
 ['0', '1jPk_y', '-1jPk_x', '0'],
 ['0', '1jPk_z', '0', '-1jPk_x']]

In [63]:
matrix_order(order_variables,Christoffel(frw,frw_con)[2],1,'small')

[['1jSa^-2k_y', '0', '1a^-1a_t +1P_t', '0'],
 ['0', '-1jPk_y', '1jPk_x', '0'],
 ['1a^-1a_t +1P_t', '1jPk_x', '1jPk_y', '1jPk_z'],
 ['0', '0', '1jPk_z', '-1jPk_y']]

In [64]:
matrix_order(order_variables,Christoffel(frw,frw_con)[3],1,'small')

[['1jSa^-2k_z', '0', '0', '1a^-1a_t +1P_t'],
 ['0', '-1jPk_z', '0', '1jPk_x'],
 ['0', '0', '-1jPk_z', '1jPk_y'],
 ['1a^-1a_t +1P_t', '1jPk_x', '1jPk_y', '1jPk_z']]

In [65]:
matrix_order(order_variables,Ricci_Tensor(frw,frw_con),1,'small')

[['-1Sa^-2k_x^2 -1Sa^-2k_y^2 -1Sa^-2k_z^2 -3a^-1a_tt -3P_tt +3S_ta^-1a_t -6P_ta^-1a_t',
  '-2jP_tk_x +2jSa^-1a_tk_x',
  '1jS_tk_y -1jP_tk_y +2jSa^-1a_tk_y',
  '-2jP_tk_z +2jSa^-1a_tk_z'],
 ['-2jP_tk_x +2jSa^-1a_tk_x',
  '2a_t^2 +1aa_tt +1P_tta^2 +6P_taa_t +4Pa_t^2 +2Paa_tt -1S_taa_t -4Sa_t^2 -2Saa_tt +2Pk_x^2 +1Pk_y^2 +1Pk_z^2 +1Sk_x^2',
  '1Pk_xk_y +1Sk_xk_y',
  '1Pk_xk_z +1Sk_xk_z'],
 ['-3jP_tk_y +2jSa^-1a_tk_y',
  '1Pk_xk_y +1Sk_xk_y',
  '2a_t^2 +1aa_tt +1P_tta^2 +6P_taa_t +4Pa_t^2 +2Paa_tt -1S_taa_t -4Sa_t^2 -2Saa_tt +1Pk_x^2 +2Pk_y^2 +1Pk_z^2 +1Sk_y^2',
  '1Pk_yk_z +1Sk_yk_z'],
 ['-2jP_tk_z +2jSa^-1a_tk_z',
  '1Pk_xk_z +1Sk_xk_z',
  '1Pk_yk_z +1Sk_yk_z',
  '2a_t^2 +1aa_tt +1P_tta^2 +6P_taa_t +4Pa_t^2 +2Paa_tt -1S_taa_t -4Sa_t^2 -2Saa_tt +1Pk_x^2 +1Pk_y^2 +2Pk_z^2 +1Sk_z^2']]

In [69]:
scalar_string0 = contract(order(order_variables,Ricci_Scalar(frw,frw_con),100000,'small'))
scalar_string0

'2Sa^-2k_x^2 -2PSa^-2k_x^2 +2Sa^-2k_y^2 +2Sa^-2k_z^2 +6a^-1a_tt +6P_tt -12PP_tt -72PP_ta^-1a_t -48P^2a^-2a_t^2 -24P^2a^-1a_tt -6S_ta^-1a_t -6P_tS_t +12PP_tS_t +24P^2S_ta^-1a_t -6S^2a^-2k_x^2 +8PS^2a^-2k_x^2 -16P^2Sa^-2k_x^2 +16P^3Sa^-2k_x^2 -6S^2a^-2k_y^2 +8PS^2a^-2k_y^2 -16P^2Sa^-2k_y^2 +16P^3Sa^-2k_y^2 -6S^2a^-2k_z^2 -16P^2Sa^-2k_z^2 +16P^3Sa^-2k_z^2 +24P_ta^-1a_t -24PP_t^2 -48P^2P_ta^-1a_t +24P^2P_t^2 +96P^3P_ta^-1a_t +96P^4a^-2a_t^2 -12Sa^-1a_tt -12P_ttS +24PP_ttS +144PP_tSa^-1a_t +96P^2Sa^-2a_t^2 +48P^2Sa^-1a_tt +32P^2S^2a^-2k_x^2 -32P^3S^2a^-2k_x^2 +32P^2S^2a^-2k_y^2 -32P^3S^2a^-2k_y^2 +32P^2S^2a^-2k_z^2 -32P^3S^2a^-2k_z^2 -48P_tSa^-1a_t +48PP_t^2S +96P^2P_tSa^-1a_t -48P^2P_t^2S -192P^3P_tSa^-1a_t -192P^4Sa^-2a_t^2 +6a^-2a_t^2 -12Sa^-2a_t^2 +4Pa^-2k_x^2 -22P^2a^-2k_x^2 +4Pa^-2k_y^2 -22P^2a^-2k_y^2 +4Pa^-2k_z^2 -22P^2a^-2k_z^2 +20P^3a^-2k_y^2 +24P^4a^-2k_y^2 +20P^3a^-2k_z^2 +24P^4a^-2k_z^2 +20P^3a^-2k_x^2 -16P^5a^-2k_y^2 -16P^5a^-2k_z^2 +24P^4a^-2k_x^2 -2PSa^-2k_y^2 -2PSa^-2k_z^2 

In [70]:
upper = 1000

scalar_string = scalar_string0

scalar_string = scalar_string.replace('S_tt','QQ1QQ')
scalar_string = scalar_string.replace('S_t', 'QQ2QQ')
scalar_string = scalar_string.replace('S',   'QQ3QQ')
scalar_string = scalar_string.replace('P_tt','QQ4QQ')
scalar_string = scalar_string.replace('P_t', 'QQ5QQ')
scalar_string = scalar_string.replace('P',   'QQ6QQ')
scalar_string = scalar_string.replace('a_tt','QQ7QQ')

scalar_string = scalar_string.replace('a^-1a_t'.format(power),'H'.format(power))

for power in range(2,upper):
    scalar_string = scalar_string.replace('a^-{0}a_t^{0}'.format(power),'H^[{}]'.format(power))
scalar_string = scalar_string.replace('[','{')  
scalar_string = scalar_string.replace(']','}')

for power in range(upper):
    scalar_string = scalar_string.replace('^{}'.format(power),'^[{}]'.format(power))
    scalar_string = scalar_string.replace('^-{}'.format(power),'^[-{}]'.format(power))
scalar_string = scalar_string.replace('[','{')  
scalar_string = scalar_string.replace(']','}')


scalar_string = scalar_string.replace('a_t', 'QQ8QQ')
scalar_string = scalar_string.replace('a',   'QQ9QQ')

scalar_string = scalar_string.replace('QQ1QQ','\Psi_{,00}')
scalar_string = scalar_string.replace('QQ2QQ','\Psi_{,0}')
scalar_string = scalar_string.replace('QQ3QQ','\Psi ')
scalar_string = scalar_string.replace('QQ4QQ','\Phi_{,00}')
scalar_string = scalar_string.replace('QQ5QQ','\Phi_{,0}')
scalar_string = scalar_string.replace('QQ6QQ','\Phi ')
scalar_string = scalar_string.replace('QQ7QQ','\ddot{a}')
scalar_string = scalar_string.replace('QQ8QQ','\dot{a}')
scalar_string = scalar_string.replace('QQ9QQ','a')

print('$$'+ scalar_string +'$$')

$$2\Psi a^{-2}k_x^{2} -2\Phi \Psi a^{-2}k_x^{2} +2\Psi a^{-2}k_y^{2} +2\Psi a^{-2}k_z^{2} +6a^{-1}\ddot{a} +6\Phi_{,00} -12\Phi \Phi_{,00} -72\Phi \Phi_{,0}H -48\Phi ^{2}H^{2} -24\Phi ^{2}a^{-1}\ddot{a} -6\Psi_{,0}H -6\Phi_{,0}\Psi_{,0} +12\Phi \Phi_{,0}\Psi_{,0} +24\Phi ^{2}\Psi_{,0}H -6\Psi ^{2}a^{-2}k_x^{2} +8\Phi \Psi ^{2}a^{-2}k_x^{2} -16\Phi ^{2}\Psi a^{-2}k_x^{2} +16\Phi ^{3}\Psi a^{-2}k_x^{2} -6\Psi ^{2}a^{-2}k_y^{2} +8\Phi \Psi ^{2}a^{-2}k_y^{2} -16\Phi ^{2}\Psi a^{-2}k_y^{2} +16\Phi ^{3}\Psi a^{-2}k_y^{2} -6\Psi ^{2}a^{-2}k_z^{2} -16\Phi ^{2}\Psi a^{-2}k_z^{2} +16\Phi ^{3}\Psi a^{-2}k_z^{2} +24\Phi_{,0}H -24\Phi \Phi_{,0}^{2} -48\Phi ^{2}\Phi_{,0}H +24\Phi ^{2}\Phi_{,0}^{2} +96\Phi ^{3}\Phi_{,0}H +96\Phi ^{4}H^{2} -12\Psi a^{-1}\ddot{a} -12\Phi_{,00}\Psi  +24\Phi \Phi_{,00}\Psi  +144\Phi \Phi_{,0}\Psi H +96\Phi ^{2}\Psi H^{2} +48\Phi ^{2}\Psi a^{-1}\ddot{a} +32\Phi ^{2}\Psi ^{2}a^{-2}k_x^{2} -32\Phi ^{3}\Psi ^{2}a^{-2}k_x^{2} +32\Phi ^{2}\Psi ^{2}a^{-2}k_y^{2} -32\Phi ^{3}\Ps

$$2\Psi a^{-2}k_x^{2} -2\Phi \Psi a^{-2}k_x^{2} +2\Psi a^{-2}k_y^{2} +2\Psi a^{-2}k_z^{2} +6a^{-1}\ddot{a} +6\Phi_{,00} -12\Phi \Phi_{,00} -72\Phi \Phi_{,0}H -48\Phi ^{2}H^{2} -24\Phi ^{2}a^{-1}\ddot{a} -6\Psi_{,0}H -6\Phi_{,0}\Psi_{,0} +12\Phi \Phi_{,0}\Psi_{,0} +24\Phi ^{2}\Psi_{,0}H -6\Psi ^{2}a^{-2}k_x^{2} +8\Phi \Psi ^{2}a^{-2}k_x^{2} -16\Phi ^{2}\Psi a^{-2}k_x^{2} +16\Phi ^{3}\Psi a^{-2}k_x^{2} -6\Psi ^{2}a^{-2}k_y^{2} +8\Phi \Psi ^{2}a^{-2}k_y^{2} -16\Phi ^{2}\Psi a^{-2}k_y^{2} +16\Phi ^{3}\Psi a^{-2}k_y^{2} -6\Psi ^{2}a^{-2}k_z^{2} -16\Phi ^{2}\Psi a^{-2}k_z^{2} +16\Phi ^{3}\Psi a^{-2}k_z^{2} +24\Phi_{,0}H -24\Phi \Phi_{,0}^{2} -48\Phi ^{2}\Phi_{,0}H +24\Phi ^{2}\Phi_{,0}^{2} +96\Phi ^{3}\Phi_{,0}H +96\Phi ^{4}H^{2} -12\Psi a^{-1}\ddot{a} -12\Phi_{,00}\Psi  +24\Phi \Phi_{,00}\Psi  +144\Phi \Phi_{,0}\Psi H +96\Phi ^{2}\Psi H^{2} +48\Phi ^{2}\Psi a^{-1}\ddot{a} +32\Phi ^{2}\Psi ^{2}a^{-2}k_x^{2} -32\Phi ^{3}\Psi ^{2}a^{-2}k_x^{2} +32\Phi ^{2}\Psi ^{2}a^{-2}k_y^{2} -32\Phi ^{3}\Psi ^{2}a^{-2}k_y^{2} +32\Phi ^{2}\Psi ^{2}a^{-2}k_z^{2} -32\Phi ^{3}\Psi ^{2}a^{-2}k_z^{2} -48\Phi_{,0}\Psi H +48\Phi \Phi_{,0}^{2}\Psi  +96\Phi ^{2}\Phi_{,0}\Psi H -48\Phi ^{2}\Phi_{,0}^{2}\Psi  -192\Phi ^{3}\Phi_{,0}\Psi H -192\Phi ^{4}\Psi H^{2} +6H^{2} -12\Psi H^{2} +4\Phi a^{-2}k_x^{2} -22\Phi ^{2}a^{-2}k_x^{2} +4\Phi a^{-2}k_y^{2} -22\Phi ^{2}a^{-2}k_y^{2} +4\Phi a^{-2}k_z^{2} -22\Phi ^{2}a^{-2}k_z^{2} +20\Phi ^{3}a^{-2}k_y^{2} +24\Phi ^{4}a^{-2}k_y^{2} +20\Phi ^{3}a^{-2}k_z^{2} +24\Phi ^{4}a^{-2}k_z^{2} +20\Phi ^{3}a^{-2}k_x^{2} -16\Phi ^{5}a^{-2}k_y^{2} -16\Phi ^{5}a^{-2}k_z^{2} +24\Phi ^{4}a^{-2}k_x^{2} -2\Phi \Psi a^{-2}k_y^{2} -2\Phi \Psi a^{-2}k_z^{2} -16\Phi ^{5}a^{-2}k_x^{2} +8\Phi \Psi ^{2}a^{-2}k_z^{2}$$

# Alcubierre Metric

$$ g_{00} = v_s^2(t)f^2(r_s(t)) -1 \hspace{3pc} g_{10}=g_{01} = -2v_s(t)f(r_s(t)) \hspace{3pc} g_{11}=g_{22}=g_{33} = 1 $$

$$ g^{00} = -\left(3v_s^2(t)f^2(r_s(t)) +1\right)^{-1} \hspace{3pc} g^{10}=g^{01} = -2v_s(t)f(r_s(t))\left(3v_s^2(t)f^2(r_s(t)) +1\right)^{-1}
\hspace{3pc} g^{11}= -\frac{1}{3} + \frac{4}{3} \left(3v_s^2(t)f^2(r_s(t)) +1\right)^{-1}  \hspace{3pc} g^{22}=g^{33} = 1 $$

$$ v \equiv v_s(t)= \frac{dx_s}{dt} \hspace{3pc} f \equiv f(r_s(t))= \frac{\tanh(\sigma(r_s+R))-\tanh(\sigma(r_s-R))}{2\tanh(\sigma R)}  \hspace{3pc} D \equiv 3v^2f^2+1 \hspace{3pc} (\det(g_{\mu\nu}))^2 = D^2$$

$$ Tp \equiv \tanh(\sigma(r_s+R)) \hspace{3pc} Tq \equiv \tanh(\sigma(r_s-R))  \hspace{3pc} To \equiv \tanh(\sigma R) $$

$$b = \frac{1}{3} \hspace{3pc} u \equiv \sigma \hspace{3pc} s \equiv x_s(t) \hspace{3pc} r \equiv r_s(t) = \sqrt{(x-x_s)^2+y^2+z^2}$$

In [71]:
#c = 1
alc0 = [['1bD - 4b', '-2vf', '0', '0'], \
        [ '-2vf', '1', '0', '0'], \
        [ '0', '0', '1', '0'], \
        [ '0', '0', '0', '1']]

alc_con0 = [['-1D^-1', '-2vfD^-1', '0', '0'], \
            [ '-2vfD^-1', '-1b + 4bD^-1', '0', '0'], \
            [ '0', '0', '1', '0'], \
            [ '0', '0', '0', '1']]

#================

r_squared = P('x^2+s^2-2sx+y^2+z^2')

Chains = {'t':{'s':'v','v':'v_t', 'v_t':'v_tt'}, 'x':{}, 'y':{}, 'z':{}}

dt_r = mult( [[-.5,['r'],[-1]]] , partial('t',r_squared) )
dx_r = mult( [[-.5,['r'],[-1]]] , partial('x',r_squared) )
dy_r = mult( [[-.5,['r'],[-1]]] , partial('y',r_squared) )
dz_r = mult( [[-.5,['r'],[-1]]] , partial('z',r_squared) )

d_Tp = [[1,['u'],[1]],[-1,['u','Tp'],[1,2]]] # d_r
d_Tq = [[1,['u'],[1]],[-1,['u','Tq'],[1,2]]] # d_r
d_To = [[1,['u'],[1]],[-1,['u','To'],[1,2]]] # d_r

dt_Tp = mult(d_Tp,dt_r) ; dt_Tq = mult(d_Tq,dt_r) ; dt_To = mult(d_To,dt_r) ; 
dx_Tp = mult(d_Tp,dx_r) ; dx_Tq = mult(d_Tq,dx_r) ; dx_To = mult(d_To,dx_r) ; 
dy_Tp = mult(d_Tp,dy_r) ; dy_Tq = mult(d_Tq,dy_r) ; dy_To = mult(d_To,dy_r) ; 
dz_Tp = mult(d_Tp,dz_r) ; dz_Tq = mult(d_Tq,dz_r) ; dz_To = mult(d_To,dz_r) ;

Tp = [[1,['Tp'],[1]]] ; Tq = [[1,['Tq'],[1]]] ; To = [[1,['To'],[1]]]

f = [[0.5,['Tp','To'],[1,-1]],[-0.5,['Tq','To'],[1,-1]]]

dt_f = mult([[0.5,['To'],[-2]]], addi([ mult(To,dt_Tp), multi([P('-1'),To,dt_Tq]), multi([P('-1'),Tp,dt_To]), mult(Tq,dt_Tp) ]) )
dx_f = mult([[0.5,['To'],[-2]]], addi([ mult(To,dx_Tp), multi([P('-1'),To,dx_Tq]), multi([P('-1'),Tp,dx_To]), mult(Tq,dx_Tp) ]) )
dy_f = mult([[0.5,['To'],[-2]]], addi([ mult(To,dy_Tp), multi([P('-1'),To,dy_Tq]), multi([P('-1'),Tp,dy_To]), mult(Tq,dy_Tp) ]) )
dz_f = mult([[0.5,['To'],[-2]]], addi([ mult(To,dz_Tp), multi([P('-1'),To,dz_Tq]), multi([P('-1'),Tp,dz_To]), mult(Tq,dz_Tp) ]) )

#================

Chains = { \
             't':{'s':'v', 'v':'v_t', 'v_t':'v_tt',\
                  'f':dt_f, 'Tp': dt_Tp, 'Tq': dt_Tq, 'To': dt_To, 'r': dt_r},\
             'x':{'f':dx_f, 'Tp': dx_Tp, 'Tq': dx_Tq, 'To': dx_To, 'r': dx_r },\
             'y':{'f':dy_f, 'Tp': dy_Tp, 'Tq': dy_Tq, 'To': dy_To, 'r': dy_r },\
             'z':{'f':dz_f, 'Tp': dz_Tp, 'Tq': dz_Tq, 'To': dz_To, 'r': dz_r } \
             }

Alc     = matrix_convert(alc0)
Alc_con = matrix_convert(alc_con0)

In [72]:
matrix_convert(Christoffel(Alc,Alc_con)[0])

[['2D^-1To^-1Tp^2fr^-1suv^3 -2D^-1To^-1Tp^2fr^-1uv^3x -2D^-1To^-1Tq^2fr^-1suv^3 +2D^-1To^-1Tq^2fr^-1uv^3x +2D^-1To^-2Tpfr^-1suv^3 -2D^-1To^-2Tpfr^-1uv^3x -2D^-1Tpfr^-1suv^3 +2D^-1Tpfr^-1uv^3x -2D^-1To^-2Tqfr^-1suv^3 +2D^-1To^-2Tqfr^-1uv^3x +2D^-1To^-2Tp^2Tqfr^-1suv^3 -2D^-1To^-2Tp^2Tqfr^-1uv^3x +4D^-1f^2vv_t',
  '0',
  '1D^-1To^-1Tp^2fr^-1uv^2y -1D^-1To^-1Tq^2fr^-1uv^2y +1D^-1To^-2Tpfr^-1uv^2y -1D^-1Tpfr^-1uv^2y -1D^-1To^-2Tqfr^-1uv^2y +1D^-1To^-2Tp^2Tqfr^-1uv^2y',
  '1D^-1To^-1Tp^2fr^-1uv^2z -1D^-1To^-1Tq^2fr^-1uv^2z +1D^-1To^-2Tpfr^-1uv^2z -1D^-1Tpfr^-1uv^2z -1D^-1To^-2Tqfr^-1uv^2z +1D^-1To^-2Tp^2Tqfr^-1uv^2z'],
 ['0',
  '1D^-1To^-1Tp^2r^-1uvx -1D^-1To^-1Tp^2r^-1suv -1D^-1To^-1Tq^2r^-1uvx +1D^-1To^-1Tq^2r^-1suv +1D^-1To^-2Tpr^-1uvx -1D^-1To^-2Tpr^-1suv -1D^-1Tpr^-1uvx +1D^-1Tpr^-1suv -1D^-1To^-2Tqr^-1uvx +1D^-1To^-2Tqr^-1suv +1D^-1To^-2Tp^2Tqr^-1uvx -1D^-1To^-2Tp^2Tqr^-1suv',
  '0.5D^-1To^-1Tp^2r^-1uvy -0.5D^-1To^-1Tq^2r^-1uvy +0.5D^-1To^-2Tpr^-1uvy -0.5D^-1Tpr^-1uvy -0.5D^-1To^-2Tqr

In [73]:
matrix_convert(Christoffel(Alc,Alc_con)[1])

[['1To^-1Tp^2br^-1suv^2 -1To^-1Tp^2br^-1uv^2x -1To^-1Tq^2br^-1suv^2 +1To^-1Tq^2br^-1uv^2x +1To^-2Tpbr^-1suv^2 -1To^-2Tpbr^-1uv^2x -1Tpbr^-1suv^2 +1Tpbr^-1uv^2x -1To^-2Tqbr^-1suv^2 +1To^-2Tqbr^-1uv^2x +1To^-2Tp^2Tqbr^-1suv^2 -1To^-2Tp^2Tqbr^-1uv^2x +2bfv_t -4D^-1To^-1Tp^2br^-1suv^2 +4D^-1To^-1Tp^2br^-1uv^2x +4D^-1To^-1Tq^2br^-1suv^2 -4D^-1To^-1Tq^2br^-1uv^2x -4D^-1To^-2Tpbr^-1suv^2 +4D^-1To^-2Tpbr^-1uv^2x +4D^-1Tpbr^-1suv^2 -4D^-1Tpbr^-1uv^2x +4D^-1To^-2Tqbr^-1suv^2 -4D^-1To^-2Tqbr^-1uv^2x -4D^-1To^-2Tp^2Tqbr^-1suv^2 +4D^-1To^-2Tp^2Tqbr^-1uv^2x -8D^-1bfv_t',
  '0',
  '0.5To^-1Tp^2br^-1uvy -0.5To^-1Tq^2br^-1uvy +0.5To^-2Tpbr^-1uvy -0.5Tpbr^-1uvy -0.5To^-2Tqbr^-1uvy +0.5To^-2Tp^2Tqbr^-1uvy -2D^-1To^-1Tp^2br^-1uvy +2D^-1To^-1Tq^2br^-1uvy -2D^-1To^-2Tpbr^-1uvy +2D^-1Tpbr^-1uvy +2D^-1To^-2Tqbr^-1uvy -2D^-1To^-2Tp^2Tqbr^-1uvy',
  '0.5To^-1Tp^2br^-1uvz -0.5To^-1Tq^2br^-1uvz +0.5To^-2Tpbr^-1uvz -0.5Tpbr^-1uvz -0.5To^-2Tqbr^-1uvz +0.5To^-2Tp^2Tqbr^-1uvz -2D^-1To^-1Tp^2br^-1uvz +2D^-1To^-1Tq^2br^

In [74]:
matrix_convert(Christoffel(Alc,Alc_con)[2])

[['0',
  '0.5To^-1Tp^2r^-1uvy -0.5To^-1Tq^2r^-1uvy +0.5To^-2Tpr^-1uvy -0.5Tpr^-1uvy -0.5To^-2Tqr^-1uvy +0.5To^-2Tp^2Tqr^-1uvy',
  '0',
  '0'],
 ['0.5To^-1Tp^2r^-1uvy -0.5To^-1Tq^2r^-1uvy +0.5To^-2Tpr^-1uvy -0.5Tpr^-1uvy -0.5To^-2Tqr^-1uvy +0.5To^-2Tp^2Tqr^-1uvy',
  '0',
  '0',
  '0'],
 ['0', '0', '0', '0'],
 ['0', '0', '0', '0']]

In [75]:
matrix_convert(Christoffel(Alc,Alc_con)[3])

[['0',
  '0.5To^-1Tp^2r^-1uvz -0.5To^-1Tq^2r^-1uvz +0.5To^-2Tpr^-1uvz -0.5Tpr^-1uvz -0.5To^-2Tqr^-1uvz +0.5To^-2Tp^2Tqr^-1uvz',
  '0',
  '0'],
 ['0.5To^-1Tp^2r^-1uvz -0.5To^-1Tq^2r^-1uvz +0.5To^-2Tpr^-1uvz -0.5Tpr^-1uvz -0.5To^-2Tqr^-1uvz +0.5To^-2Tp^2Tqr^-1uvz',
  '0',
  '0',
  '0'],
 ['0', '0', '0', '0'],
 ['0', '0', '0', '0']]

In [76]:
matrix_convert(Ricci_Tensor(Alc,Alc_con))

[['2To^-2Tp^2br^-2su^2v^2x -1To^-2Tp^2br^-2s^2u^2v^2 -4Tp^2br^-2su^2v^2x +2Tp^2br^-2s^2u^2v^2 -8To^-1Tpbr^-2su^2v^2x +4To^-1Tpbr^-2s^2u^2v^2 +4To^-1Tp^3br^-2su^2v^2x -2To^-1Tp^3br^-2s^2u^2v^2 +2To^-1Tp^2br^-3suv^2x -1To^-1Tp^2br^-3s^2uv^2 -1To^-2Tp^2br^-2u^2v^2x^2 +2Tp^2br^-2u^2v^2x^2 +4To^-1Tpbr^-2u^2v^2x^2 -2To^-1Tp^3br^-2u^2v^2x^2 -1To^-1Tp^2br^-3uv^2x^2 -1To^-1Tp^2br^-1uv^2 -4To^-2Tq^2br^-2su^2v^2x +2To^-2Tq^2br^-2s^2u^2v^2 +2Tq^2br^-2su^2v^2x -1Tq^2br^-2s^2u^2v^2 +8To^-1Tqbr^-2su^2v^2x -4To^-1Tqbr^-2s^2u^2v^2 -4To^-1Tq^3br^-2su^2v^2x +2To^-1Tq^3br^-2s^2u^2v^2 -2To^-1Tq^2br^-3suv^2x +1To^-1Tq^2br^-3s^2uv^2 +2To^-2Tq^2br^-2u^2v^2x^2 -1Tq^2br^-2u^2v^2x^2 -4To^-1Tqbr^-2u^2v^2x^2 +2To^-1Tq^3br^-2u^2v^2x^2 +1To^-1Tq^2br^-3uv^2x^2 +1To^-1Tq^2br^-1uv^2 +4To^-3Tpbr^-2su^2v^2x -2To^-3Tpbr^-2s^2u^2v^2 +2To^-2Tpbr^-3suv^2x -1To^-2Tpbr^-3s^2uv^2 -2To^-3Tpbr^-2u^2v^2x^2 -1To^-2Tpbr^-3uv^2x^2 -1To^-2Tpbr^-1uv^2 +2br^-2su^2v^2x -1br^-2s^2u^2v^2 -2Tpbr^-3suv^2x +1Tpbr^-3s^2uv^2 -1br^-2u^2v^2x^2 +1

In [77]:
warp_scalar = Ricci_Scalar(Alc,Alc_con)
contract(warp_scalar)

'-4D^-1To^-2Tp^2br^-2su^2v^2x +2D^-1To^-2Tp^2br^-2s^2u^2v^2 +8D^-1Tp^2br^-2su^2v^2x -4D^-1Tp^2br^-2s^2u^2v^2 +16D^-1To^-1Tpbr^-2su^2v^2x -8D^-1To^-1Tpbr^-2s^2u^2v^2 -8D^-1To^-1Tp^3br^-2su^2v^2x +4D^-1To^-1Tp^3br^-2s^2u^2v^2 -4D^-1To^-1Tp^2br^-3suv^2x +2D^-1To^-1Tp^2br^-3s^2uv^2 +2D^-1To^-2Tp^2br^-2u^2v^2x^2 -4D^-1Tp^2br^-2u^2v^2x^2 -8D^-1To^-1Tpbr^-2u^2v^2x^2 +4D^-1To^-1Tp^3br^-2u^2v^2x^2 +2D^-1To^-1Tp^2br^-3uv^2x^2 +2D^-1To^-1Tp^2br^-1uv^2 +8D^-1To^-2Tq^2br^-2su^2v^2x -4D^-1To^-2Tq^2br^-2s^2u^2v^2 -4D^-1Tq^2br^-2su^2v^2x +2D^-1Tq^2br^-2s^2u^2v^2 -16D^-1To^-1Tqbr^-2su^2v^2x +8D^-1To^-1Tqbr^-2s^2u^2v^2 +8D^-1To^-1Tq^3br^-2su^2v^2x -4D^-1To^-1Tq^3br^-2s^2u^2v^2 +4D^-1To^-1Tq^2br^-3suv^2x -2D^-1To^-1Tq^2br^-3s^2uv^2 -4D^-1To^-2Tq^2br^-2u^2v^2x^2 +2D^-1Tq^2br^-2u^2v^2x^2 +8D^-1To^-1Tqbr^-2u^2v^2x^2 -4D^-1To^-1Tq^3br^-2u^2v^2x^2 -2D^-1To^-1Tq^2br^-3uv^2x^2 -2D^-1To^-1Tq^2br^-1uv^2 -8D^-1To^-3Tpbr^-2su^2v^2x +4D^-1To^-3Tpbr^-2s^2u^2v^2 -4D^-1To^-2Tpbr^-3suv^2x +2D^-1To^-2Tpbr^-3s^2uv^2 +4D^-

In [78]:
warp_simplified = warp_scalar
warp_simplified = substitute(mult(P('D^2'),warp_simplified),'D',P('3v^2f^2')) #factor out a 1/D^2
warp_simplified = substitute(warp_simplified,'f',f) #replace F with (Tp-Tq)/2To
warp_simplified = multi([P('r^-2'),r_squared,warp_simplified]) # replace r^2
warp_simplified = substitute(warp_simplified,'b',P('{}'.format(1/3))) #replace b with 1/3

print ('1/D^2 * ...')
print(contract(warp_simplified))

KeyboardInterrupt: 

# Same but keeping in terms of $ \frac{df}{dr_s} = F \hspace{3pc} (\det(g_{\mu\nu}))^2 = D^2$

In [79]:
#c = 1
alc0 = [['1bD - 4b', '-2vf', '0', '0'], \
        [ '-2vf', '1', '0', '0'], \
        [ '0', '0', '1', '0'], \
        [ '0', '0', '0', '1']]

alc_con0 = [['-1D^-1', '-2vfD^-1', '0', '0'], \
            [ '-2vfD^-1', '-1b + 4bD^-1', '0', '0'], \
            [ '0', '0', '1', '0'], \
            [ '0', '0', '0', '1']]

#================

r_squared = P('x^2+s^2-2sx+y^2+z^2')

Chains = {'t':{'s':'v','v':'v_t', 'v_t':'v_tt'}, 'x':{}, 'y':{}, 'z':{}}

dt_r = mult( [[-.5,['r'],[-1]]] , partial('t',r_squared) )
dx_r = mult( [[-.5,['r'],[-1]]] , partial('x',r_squared) )
dy_r = mult( [[-.5,['r'],[-1]]] , partial('y',r_squared) )
dz_r = mult( [[-.5,['r'],[-1]]] , partial('z',r_squared) )

dt_f = mult(dt_r,[[1,['F'],[1]]])
dx_f = mult(dx_r,[[1,['F'],[1]]])
dy_f = mult(dy_r,[[1,['F'],[1]]])
dz_f = mult(dz_r,[[1,['F'],[1]]])

#================

Chains = { \
             't':{'s':'v', 'v':'v_t', 'v_t':'v_tt',\
                  'f':dt_f},\
             'x':{'f':dx_f},\
             'y':{'f':dy_f},\
             'z':{'f':dz_f} \
             }

Alc     = matrix_convert(alc0)
Alc_con = matrix_convert(alc_con0)

In [80]:
matrix_convert(Christoffel(Alc,Alc_con)[0])

[['-4D^-1Ffr^-1sv^3 +4D^-1Ffr^-1v^3x +4D^-1f^2vv_t',
  '0',
  '-2D^-1Ffr^-1v^2y',
  '-2D^-1Ffr^-1v^2z'],
 ['0', '-2D^-1Fr^-1vx +2D^-1Fr^-1sv', '-1D^-1Fr^-1vy', '-1D^-1Fr^-1vz'],
 ['-2D^-1Ffr^-1v^2y', '-1D^-1Fr^-1vy', '0', '0'],
 ['-2D^-1Ffr^-1v^2z', '-1D^-1Fr^-1vz', '0', '0']]

In [81]:
matrix_convert(Christoffel(Alc,Alc_con)[1])

[['-2Fbr^-1sv^2 +2Fbr^-1v^2x +2bfv_t +8D^-1Fbr^-1sv^2 -8D^-1Fbr^-1v^2x -8D^-1bfv_t',
  '0',
  '-1Fbr^-1vy +4D^-1Fbr^-1vy',
  '-1Fbr^-1vz +4D^-1Fbr^-1vz'],
 ['0',
  '-4D^-1Ffr^-1v^2x +4D^-1Ffr^-1sv^2',
  '-2D^-1Ffr^-1v^2y',
  '-2D^-1Ffr^-1v^2z'],
 ['-1Fbr^-1vy +4D^-1Fbr^-1vy', '-2D^-1Ffr^-1v^2y', '0', '0'],
 ['-1Fbr^-1vz +4D^-1Fbr^-1vz', '-2D^-1Ffr^-1v^2z', '0', '0']]

In [82]:
matrix_convert(Christoffel(Alc,Alc_con)[2])

[['0', '-1Fr^-1vy', '0', '0'],
 ['-1Fr^-1vy', '0', '0', '0'],
 ['0', '0', '0', '0'],
 ['0', '0', '0', '0']]

In [83]:
matrix_convert(Christoffel(Alc,Alc_con)[3])

[['0', '-1Fr^-1vz', '0', '0'],
 ['-1Fr^-1vz', '0', '0', '0'],
 ['0', '0', '0', '0'],
 ['0', '0', '0', '0']]

In [84]:
R_Tensor = Ricci_Tensor(Alc,Alc_con)
matrix_convert(R_Tensor)

[['2Fbr^-1v^2 -2Fbr^-1v_tx +2Fbr^-1sv_t -8D^-1Fbr^-1v^2 +8D^-1Fbr^-1v_tx -8D^-1Fbr^-1sv_t +16D^-1F^2bfr^-2sv^4x -8D^-1F^2bfr^-2v^4x^2 -8D^-1Fbf^2r^-1v^2v_tx -64D^-2F^2bfr^-2sv^4x +32D^-2F^2bfr^-2v^4x^2 +32D^-2Fbf^2r^-1v^2v_tx -8D^-1F^2bfr^-2s^2v^4 +8D^-1Fbf^2r^-1sv^2v_t +32D^-2F^2bfr^-2s^2v^4 -32D^-2Fbf^2r^-1sv^2v_t -2F^2br^-2v^2y^2 +8D^-1F^2br^-2v^2y^2 -2F^2br^-2v^2z^2 +8D^-1F^2br^-2v^2z^2',
  '-2Fr^-1v -8D^-1F^2r^-2sv^3x +4D^-1F^2r^-2s^2v^3 +4D^-1F^2r^-2v^3x^2 -4D^-1Ffr^-1v^3 +8D^-1Ffr^-1vv_tx -8D^-1Ffr^-1svv_t -8D^-1F^2br^-2sv^3x +4D^-1F^2br^-2v^3x^2 +4D^-1Fbfr^-1vv_tx +32D^-2F^2br^-2sv^3x -16D^-2F^2br^-2v^3x^2 -16D^-2Fbfr^-1vv_tx +4D^-1F^2br^-2s^2v^3 -4D^-1Fbfr^-1svv_t -16D^-2F^2br^-2s^2v^3 +16D^-2Fbfr^-1svv_t',
  '-2D^-1F^2r^-2sv^3y +2D^-1F^2r^-2v^3xy +4D^-1Ffr^-1vv_ty +4D^-1F^2bfr^-2v^3xy -16D^-2F^2bfr^-2v^3xy -4D^-1F^2bfr^-2sv^3y +16D^-2F^2bfr^-2sv^3y -2D^-1F^2br^-2sv^3y +2D^-1F^2br^-2v^3xy +2D^-1Fbfr^-1vv_ty +8D^-2F^2br^-2sv^3y -8D^-2F^2br^-2v^3xy -8D^-2Fbfr^-1vv_ty',
  '-2D^-1

In [85]:
warp_scalar = Ricci_Scalar(Alc,Alc_con)
contract(warp_scalar)

'-4D^-1Fbr^-1v^2 +4D^-1Fbr^-1v_tx -4D^-1Fbr^-1sv_t +16D^-2Fbr^-1v^2 -16D^-2Fbr^-1v_tx +16D^-2Fbr^-1sv_t +2D^-1F^2br^-2v^2y^2 -8D^-2F^2br^-2v^2y^2 +2D^-1F^2br^-2v^2z^2 -8D^-2F^2br^-2v^2z^2 +16D^-1Ffr^-1v^2 +32D^-2F^2fr^-2sv^4x -16D^-2F^2fr^-2s^2v^4 -16D^-2F^2fr^-2v^4x^2 +16D^-2Ff^2r^-1v^4 -32D^-2Ff^2r^-1v^2v_tx +32D^-2Ff^2r^-1sv^2v_t -4D^-1F^2r^-2v^2y^2 -8D^-2F^2f^2r^-2v^4y^2 -4D^-1F^2r^-2v^2z^2 -8D^-2F^2f^2r^-2v^4z^2'

In [86]:
warp_simplified = warp_scalar
warp_simplified = substitute(mult(P('D^2'),warp_simplified),'D',P('3v^2f^2')) #factor out a 1/D^2
warp_simplified = mult(P('r^2'),warp_simplified) #factor out a 1/r^2
warp_simplified = substitute(warp_simplified,'b',P('{}'.format(1/3))) #replace b with 1/3

print ('1/(rD)^2 * (' + contract(warp_simplified) +')')
print()

1/(rD)^2 * (12Ff^2rv^4 -28Ff^2rv^2v_tx +28Ff^2rsv^2v_t +5.333333333333333Frv^2 -5.333333333333333Frv_tx +5.333333333333333Frsv_t -18F^2f^2v^4y^2 -2.6666666666666665F^2v^2y^2 -18F^2f^2v^4z^2 -2.6666666666666665F^2v^2z^2 +48Ff^3rv^4 +32F^2fsv^4x -16F^2fs^2v^4 -16F^2fv^4x^2)



In [87]:
E_Tensor = Einstein_Tensor(R_Tensor,warp_scalar,Alc)
matrix_convert(E_Tensor)

[['2Fbr^-1v^2 -2Fbr^-1v_tx +2Fbr^-1sv_t -8D^-1Fbr^-1v^2 +8D^-1Fbr^-1v_tx -8D^-1Fbr^-1sv_t +8D^-1Fbf^2r^-1v^2v_tx -32D^-2Fbf^2r^-1v^2v_tx -8D^-1Fbf^2r^-1sv^2v_t +32D^-2Fbf^2r^-1sv^2v_t +2Fb^2r^-1v^2 -2Fb^2r^-1v_tx +2Fb^2r^-1sv_t -16D^-1Fb^2r^-1v^2 +16D^-1Fb^2r^-1v_tx -16D^-1Fb^2r^-1sv_t -1F^2b^2r^-2v^2y^2 +8D^-1F^2b^2r^-2v^2y^2 -1F^2b^2r^-2v^2z^2 +8D^-1F^2b^2r^-2v^2z^2 -8Fbfr^-1v^2 -8D^-1Fbf^2r^-1v^4 +4D^-1F^2bf^2r^-2v^4y^2 +4D^-1F^2bf^2r^-2v^4z^2 +32D^-2Fb^2r^-1v^2 -32D^-2Fb^2r^-1v_tx +32D^-2Fb^2r^-1sv_t -16D^-2F^2b^2r^-2v^2y^2 -16D^-2F^2b^2r^-2v^2z^2 +32D^-1Fbfr^-1v^2 +32D^-2Fbf^2r^-1v^4 -16D^-2F^2bf^2r^-2v^4y^2 -16D^-2F^2bf^2r^-2v^4z^2',
  '-2Fr^-1v -8D^-1F^2r^-2sv^3x +4D^-1F^2r^-2s^2v^3 +4D^-1F^2r^-2v^3x^2 -4D^-1Ffr^-1v^3 +8D^-1Ffr^-1vv_tx -8D^-1Ffr^-1svv_t -8D^-1F^2br^-2sv^3x +4D^-1F^2br^-2v^3x^2 +8D^-1Fbfr^-1vv_tx +32D^-2F^2br^-2sv^3x -16D^-2F^2br^-2v^3x^2 -32D^-2Fbfr^-1vv_tx +4D^-1F^2br^-2s^2v^3 -8D^-1Fbfr^-1svv_t -16D^-2F^2br^-2s^2v^3 +32D^-2Fbfr^-1svv_t -4D^-1Fbfr^-1v^3 +16D^-2

In [88]:
trace = addi([E_Tensor[0][0],E_Tensor[1][1],E_Tensor[2][2],E_Tensor[3][3]])

trace_simplified = trace
trace_simplified = substitute(mult(P('D^2'),trace_simplified),'D',P('3v^2f^2')) #factor out a 1/D^2
trace_simplified = mult(P('r^2'),trace_simplified) #factor out a 1/r^2
trace_simplified = substitute(trace_simplified,'b',P('{}'.format(1/3))) #replace b with 1/3

print ('1/(rD)^2 * (' + contract(trace_simplified) +')')
print()

1/(rD)^2 * (-14.666666666666668Ff^2rv^4 +30.666666666666668Ff^2rv^2v_tx -30.666666666666668Ff^2rsv^2v_t +3F^2f^4v^6y^2 -3.666666666666666F^2f^2v^4y^2 +3F^2f^4v^6z^2 -3.666666666666666F^2f^2v^4z^2 -24Ff^5rv^6 -4.444444444444445Frv^2 +4.444444444444445Frv_tx -4.444444444444445Frsv_t +4.888888888888888F^2v^2y^2 +4.888888888888888F^2v^2z^2 -16Ff^3rv^4 -32F^2fsv^4x +16F^2fs^2v^4 +16F^2fv^4x^2)

